In [1]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
def linear_congruential_generator(N, seed = 1):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = seed

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples

## Test Script:
# N = 10
# print(linear_congruential_generator(N))

In [3]:
def marsaglia_bray(seed=1):
    X = 2  # set an initial value so that the while loop is entered
    while X > 1:
        u = linear_congruential_generator(2, seed)
        u1, u2 = u[0], u[1]
        
        u1 = 2 * u1 - 1
        u2 = 2 * u2 - 1
        X = u1**2 + u2**2

        seed += 1
    
    Y = math.sqrt(-2 * math.log(X) / X)
    Z1 = u1 * Y
    Z2 = u2 * Y
    
    return Z1, Z2, seed

## Test Script:
# print(marsaglia_bray())

In [4]:
def marsaglia_bray_N(N):

    ## Generate an empty array for the generated values.
    results = np.empty(N, dtype=float)
    index = 0
    seed = 1

    while index < N:
        Z1, Z2, seed = marsaglia_bray(seed)
        
        ## Add the generated values to the results array.
        results[index] = Z1
        index += 1
        
        ## If there's still space, add the second generated value.
        if index < N:
            results[index] = Z2
            index += 1

    return results

## Test Script:
# N = 100_000
# plt.hist(marsaglia_bray_N(N), bins=20)

In [5]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

In [6]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

## Test Script:
# S = 100
# K = 100
# T = 1
# r = 0.03
# q = 0.03
# sigma = 0.2

# print("Call Option Value:", bs_call(S, K, T, r, q, sigma))
# print("Put Option Value:", bs_put(S, K, T, r, q, sigma))
# print("Call Option Delta:", bs_call_delta(S, K, T, r, q, sigma))
# print("Put Option Delta:", bs_put_delta(S, K, T, r, q, sigma))
# print("Option Gamma:", bs_gamma(S, K, T, r, q, sigma))
# print("Option Vega:", bs_vega(S, K, T, r, q, sigma))
## Verified results are correct here: https://goodcalculators.com/black-scholes-calculator/

In [7]:
def HW2_NUM1_CV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)
    ##plt.hist(Si_array, bins=10)
    
    ## Vectorized calculation for put payoff.
    Vi_array = put_payoff(Si_array, K, T, r)
    ##plt.hist(Vi_array, bins=10)

    ## Aggregation.
    S_hat = np.mean(Si_array)
    V_hat = np.mean(Vi_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_array - S_hat) * (Vi_array - V_hat)) / np.sum((Si_array - S_hat)**2)
    Wi_array = Vi_array - b_hat * (Si_array - np.exp(r * T) * S0)
    V_cv_hat = np.mean(Wi_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV_hat(N):", V_cv_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cv_hat))
    print("\n")

In [8]:
## Table values for control variate technique.

## Given values.
S0 = 56
K = 54
sigma = 0.27
q = 0
r = 0.02
T = 0.75
N = 10_000 * (2**9)

z = marsaglia_bray_N(N)

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV_hat(N): -0.08829437565900962
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 3.8893662594858456


Results for N = 20000
V_CV_hat(N): 1.11662504911072
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 2.684446834716116


Results for N = 40000
V_CV_hat(N): 3.236087526341488
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.5649843574853479


Results for N = 80000
V_CV_hat(N): 4.003252952564317
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.20218106873748098


Results for N = 160000
V_CV_hat(N): 3.9601816612608234
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.15910977743398735


Results for N = 320000
V_CV_hat(N): 3.8950360541609865
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.0939641703341505


Results for N = 640000
V_CV_hat(N): 3.809412150533354
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.008340266706517774


Results for N = 1280000
V_CV_hat(N): 3.8126782297052864
V_BS: 3.801071883826836
|V_BS - V_CV_hat(N)|: 0.01160634587845033


Results for N = 2560000
V_

In [9]:
def HW2_NUM1_AV(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si1_array = simulated_spot(S0, T, r, q, sigma, z)
    Si2_array = simulated_spot(S0, T, r, q, sigma, -z)
    
    ## Antithetic Variables Technique.
    Vi1_array = put_payoff(Si1_array, K, T, r)
    Vi2_array = put_payoff(Si2_array, K, T, r)
    V_av_hat = (np.mean(Vi1_array) + np.mean(Vi2_array)) / 2

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_AV_hat(N):", V_av_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_AV_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_av_hat))
    print("\n")

In [10]:
## Table values for anthithetic variables technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_AV(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_AV_hat(N): 1.986764384563329
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 1.814307499263507


Results for N = 20000
V_AV_hat(N): 2.7898757363983395
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 1.0111961474284965


Results for N = 40000
V_AV_hat(N): 3.751961837100339
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.0491100467264971


Results for N = 80000
V_AV_hat(N): 3.9635384125716886
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.16246652874485257


Results for N = 160000
V_AV_hat(N): 3.920028218358468
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.11895633453163201


Results for N = 320000
V_AV_hat(N): 3.860576097441551
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.05950421361471481


Results for N = 640000
V_AV_hat(N): 3.7984789694442957
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.002592914382540279


Results for N = 1280000
V_AV_hat(N): 3.810458439678693
V_BS: 3.801071883826836
|V_BS - V_AV_hat(N)|: 0.009386555851857104


Results for N = 2560000
V_

In [11]:
def HW2_NUM1_MM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_MM_hat(N):", V_mm_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_mm_hat))
    print("\n")

In [12]:
## Table values for moment matching technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_MM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_MM_hat(N): 1.1933526453633463
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 2.6077192384634897


Results for N = 20000
V_MM_hat(N): 1.8240470097149772
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 1.9770248741118588


Results for N = 40000
V_MM_hat(N): 3.0179216460523652
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.7831502377744708


Results for N = 80000
V_MM_hat(N): 3.993431056009634
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.19235917218279797


Results for N = 160000
V_MM_hat(N): 3.946883828139249
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.14581194431241284


Results for N = 320000
V_MM_hat(N): 3.8781502112108006
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.07707832738396458


Results for N = 640000
V_MM_hat(N): 3.793159187329982
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.00791269649685411


Results for N = 1280000
V_MM_hat(N): 3.812279758797451
V_BS: 3.801071883826836
|V_BS - V_MM_hat(N)|: 0.011207874970614817


Results for N = 2560000
V

In [13]:
def HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    Si_array = simulated_spot(S0, T, r, q, sigma, z)

    ## Aggregation.
    S_hat = np.mean(Si_array)

    ## Moment matching technique.
    Si_tilda_array = (np.exp(r*T)*S0/S_hat) * Si_array
    Vi_tilda_array = put_payoff(Si_tilda_array, K, T, r)
    V_mm_hat = np.mean(Vi_tilda_array)

    ## Control Variate Technique.
    b_hat = np.sum((Si_tilda_array - np.exp(r*T)*S0) * (Vi_tilda_array - V_mm_hat)) / np.sum((Si_tilda_array - np.exp(r*T)*S0)**2)
    Wi_array = Vi_tilda_array - b_hat * (Si_tilda_array - np.exp(r * T) * S0)
    V_cvmm_hat = np.mean(Wi_array)
    
    ## Output results:
    N = len(z)
    print("Results for N =", N)
    print("V_CV,MM_hat(N):", V_mm_hat)
    print("V_BS:", bs_put(S0, K, T, r, q, sigma))
    print("|V_BS - V_CV,MM_hat(N)|:", abs(bs_put(S0, K, T, r, q, sigma) - V_cvmm_hat))
    print("\n")

In [14]:
## Table values for simultaneous moment matching and control variates technique.

for k in range(10):
    end_index = 10_000 * (2**k)
    HW2_NUM1_CVMM(S0, K, T, r, q, sigma, z[:end_index])

Results for N = 10000
V_CV,MM_hat(N): 1.1933526453633463
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 2.6077192384634875


Results for N = 20000
V_CV,MM_hat(N): 1.8240470097149772
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 1.9770248741118595


Results for N = 40000
V_CV,MM_hat(N): 3.0179216460523652
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.7831502377744721


Results for N = 80000
V_CV,MM_hat(N): 3.993431056009634
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.19235917218279797


Results for N = 160000
V_CV,MM_hat(N): 3.946883828139249
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.14581194431241284


Results for N = 320000
V_CV,MM_hat(N): 3.8781502112108006
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.07707832738396236


Results for N = 640000
V_CV,MM_hat(N): 3.793159187329982
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 0.00791269649685633


Results for N = 1280000
V_CV,MM_hat(N): 3.812279758797451
V_BS: 3.801071883826836
|V_BS - V_CV,MM_hat(N)|: 